# DATA PREPARATION
We need to format the suc3.0 data, which exist in an xml tree, in such a way that ALBERT can accept it.

Because of my familiarity with dataframes, I'll begin by converting the xml tree into a pandas dataframe


I'd recommend this link for an easy read on how to use BERT for NER, which should be a good enough paralell for ALBERT
https://www.vamvas.ch/bert-for-ner/

In [ ]:
import pandas as pd
import seaborn as sns
import xml.etree.ElementTree as et 
import matplotlib.pyplot as plt
import numpy as np

### Setup

In [ ]:
# The attributes at each level of the corpus

TEXT_attributes = {
 'text_blingbring': '|',
 'text_id': '|',
 'text_lix': '|',
 'text_nk': '|',
 'text_ovix': '|',
 'text_swefn': '|',
 'text_index': 0}

SENTENCE_attributes = {
 'sentence__geocontext': '|',
 'sentence_id': '|',
 'sentence_index': 0}

WORD_attributes = {
 'word_blingbring': '|',
 'word_complemgram': '|',
 'word_compwf': '|',
 'word_dephead': '|',
 'word_deprel': '|',
 'word_lemma': '|:|',
 'word_lex': '|',
 'word_msd': '|',
 'word_pos': '|',
 'word_prefix': '|',
 'word_ref': '|',
 'word_sense': '|',
 'word_suffix': '|',
 'word_swefn': '|',
 'word_ex': '|',
 'word_name': '|',
 'word_subtype': '|',
 'word_type': '|',
 'word_index': 0}

# A dict containing all attributes associated with a single word, increading higher-level ones
WORD_LEVEL_MASTER_DICT = {}
WORD_LEVEL_MASTER_DICT.update(TEXT_attributes)
WORD_LEVEL_MASTER_DICT.update(SENTENCE_attributes)
WORD_LEVEL_MASTER_DICT.update(WORD_attributes)

In [ ]:
"""
Some tags/dictionaries have a conflicting namespace for the attributes
This function takes the attributes and adds a prefix to the keynames, to make sure nothing overwrites anything else
Returns the same dict but with updated key names
"""
def rename_attributes(dictionary, prefix):
    new_dict = {}
    for key in dictionary.attrib:
        value = dictionary.get(key)
        new_attrib_name = prefix + "_" + key
        new_dict.update({new_attrib_name:value})
        
    return new_dict

### Convert XML to Dataframe

In [ ]:
#Create a tree
xtree = et.parse("../data/suc3.xml")

#Get its root element
suc3 = xtree.getroot()

#Get all children - the texts that we will extract words from
texts = suc3.getchildren()

text_index = 0
sentence_index = 0
word_index = 0


#Create a list of datapoints. Each datapoint will be a word with all the attributes of the word itself, the attributes of the sentence it is part of, and the attributes of the text it is part of.
datapoints = []

# Loop over the texts in the corpus...
for text in texts:
    #Assign an index based on the order it appears in the corpus
    text_index = text_index + 1
    
    # Create a dict of attributes that won't cause a namespace conflict
    text_attributes = rename_attributes(text, 'text')
    
    #Add the index as an attribute
    text_attributes.update({"text_index": text_index})
    
    #Get all the sentences that make up this text
    sentences = text.getchildren()  
    
    for sentence in sentences:
        sentence_index = sentence_index + 1

        sentence_attributes = rename_attributes(sentence, 'sentence')
        sentence_attributes.update({"sentence_index": sentence_index})
   
        words = sentence.getchildren()
        
        for word in words:
            word_index = word_index + 1
            
            
            #For words, there are multiple kinds of attributes depending on the kind of word.
            # WORD_attributes is simply a list of all these attributes joined into one.
            word_attributes = WORD_attributes.copy()
            word_attributes.update({"word_index": word_index})
            
            attributes = rename_attributes(word, 'word')
            
            if word.tag == 'ne':
                attributes.update({"word_tag": "ne"})
            else:
                attributes.update({"word_tag": "w"})
                
                #If it's not a named entity, the text is not stored as an attribute but actual text. We treat it as an attribute for coherence
                attributes.update({"word_name": word.text})
                
            word_attributes.update(attributes)
            
            #Copy the master dict and fill it in with all the information we've extracted from the three loops
            WORD_LEVEL_DICT = WORD_LEVEL_MASTER_DICT.copy()
            
            WORD_LEVEL_DICT.update(text_attributes)
            WORD_LEVEL_DICT.update(sentence_attributes)
            WORD_LEVEL_DICT.update(word_attributes)
            
            datapoints.append(WORD_LEVEL_DICT)
                
print("Done")

## Replace | tokens with NaN

In [ ]:
# Having extracted all words and their associated information, let's convert it into a dataframe
df = pd.DataFrame(datapoints)

#Blingbring looks less interesting
df = df.drop(columns={'text_blingbring'})

#Replace the | with None
df = df.replace({'|': np.nan})
df.head(1)

In [ ]:
# Pickle the data
#df.to_pickle('../data/suc3_dataframe')
df = pd.read_pickle('../data/suc3_dataframe')

## Extract only the parts relevant for NER with ALBERT

In [ ]:
#Starting out, I wasn't certain which fields I was going to need, so I grabbed them all.
#Now we narrow things down
data = df[['sentence_index', 'word_name', 'word_pos', 'word_type', 'word_subtype']]
data.head(5)

## Replace the Nan type of non-entities with O

In [ ]:
# The example uses O instaed of Nan, so we follow them
data[['word_type']] = data[['word_type']].replace(np.nan,'O')
data.head(5)

## Switch to wordpiece representation of words using the albert tokenizer

Wordpiece segmentation has some advantages. Even when a model is pretrained and has a good starting point from transfer learning, some words are just too rare to represent correctly.
Wordpiece segmentation breaks unknown words down into smaller subparts

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("KB/albert-base-swedish-cased-alpha")

#An example of wordpiece segmentation. The whole word might be unknown, but splitting the word into known parts may allow the model to leverage its understanding anyway.
tokenizer.tokenize('Sovjetledarens')

In [ ]:
# Do it for all the data. This takes a while.
# Each word gets represented as a list of the parts it was split into
data['wordpiece'] = data.apply(lambda x: tokenizer.tokenize(x.word_name), axis = 1)

## Switch to BIO representation of labels

In [ ]:
def generate_bio_labels(dataframe_row):
    """
    Switch from single-word single-label to word-piece -bio-piece-labels
    This is done by, for each word, taking the list of subwords it was split into, and matching labels to that.
    We will unravel the lists later
    """
    length_of_split = len(dataframe_row.wordpiece)
    current_tag = dataframe_row.word_type

    labelpiece = []

    #The first part of the split words gets tagged with an "B-" for beginning
    labelpiece.append("B-" + current_tag)

    for i in range(1, length_of_split):   
        #The rest of the parts get tagged with an "I-" for intermediate
        labelpiece.append("I-" + current_tag)

    return labelpiece

In [ ]:
#Apply the above function to all the data
data['labelpiece'] = data.apply(lambda x: generate_bio_labels(x), axis = 1)
data.head(20)

## Delete all nans in the data

In [ ]:
# We have obsereved errors caused by the word name \n in the data, so we ignore all those rows
rows_with_errors = data[data['word_name'] == '\n'].shape[0]
total_rows = data.shape[0]
print(f"Dropping all tokens consiting of '/\ n' as these are incorrectly tagged. We lose {rows_with_errors/total_rows} percent of tokens this way")

data = data[data['word_name'] != '\n']

In [ ]:
# Let's have a look at a random sentence
data[data['sentence_index'] == 22]

## Group tokens and labels by sentence index. Add special tokens
The special tokens are [CLS] and [SEP], used as the very first and last token in a sentence. [CLS], token1, token2... tokenn, [SEP]

Their use seem to be limited in NER - CLS for example will be used to store an embedding for the whole sentence - but let's add them anyway since everyone else does.

In [ ]:
# It seems we're only gonna need the wordpieces and labelpieces
subset = data[['sentence_index','wordpiece', 'labelpiece']].copy()

#Group by sentence
sentence_groupings = subset.groupby('sentence_index')

In [ ]:
sentence_groups = []

#Get each group and put it into a list instead of a dict, for my own convenience
for sentence_index in sentence_groupings.groups.keys():
    # Get the data
    sentence_group = sentence_groupings.get_group(sentence_index)
    sentence_groups.append(sentence_group)

In [ ]:
#We obtain one element for each sentence. This element is the word-piece sentence
#And of course, a corresponding label
# This takes a while
wordpiece_sentences = []
wordpiece_labels = []

for sentence in sentence_groups:

    tokenized_sentence = []
    tokenized_labels = []
    
    #Add CLS token
    tokenized_sentence.append('[CLS]')
    tokenized_labels.append('[CLS]')

    for word in range(sentence.shape[0]):
        row = sentence.iloc[word]
        tokenized_labels.extend(row.labelpiece)
        tokenized_sentence.extend(row.wordpiece)
    
    #Add SEP token
    tokenized_sentence.append('[SEP]')
    tokenized_labels.append('[SEP]')
    
    wordpiece_sentences.append(tokenized_sentence)
    wordpiece_labels.append(tokenized_labels)


### check for errors

In [ ]:
# Count the length of all sequences and corresponding labels, they should match
error_sentences = []
for i in range(len(wordpiece_labels)):
    labelcount = len(wordpiece_labels[i])
    sentencecount = len(wordpiece_sentences[i])
    if labelcount != sentencecount:
        error_sentences.append(i)
        

print(f"There are length errors in {len(error_sentences)/len(wordpiece_labels)} percent of sentences")

In [ ]:
def look_at_error_index(index, error_sentences):
    sentence_to_look_at = index
    error_index = error_sentences[sentence_to_look_at]

    print(wordpiece_sentences[error_index])
    print(wordpiece_labels[error_index])
    # What is actually wrong?
    for i in range(len(wordpiece_sentences[error_index])):
        print(wordpiece_sentences[error_index][i] + " : " + wordpiece_labels[error_index][i])

    #Look at the original sentences
    data[data['sentence_index'] == error_index +1]


In [ ]:
# Notes on observed errors
# \n has wordtype person (sentence 55). We go back to fix that

## Convert to dataframes, store for later use

In [ ]:
# Move to a dataframe, since those pleas me
wordpiece_sentences_df = pd.DataFrame(wordpiece_sentences)
wordpiece_labels_df = pd.DataFrame(wordpiece_labels)

In [ ]:
# Store them for later use
wordpiece_sentences_df.to_pickle('../data/wordpiece_sentences_df')
wordpiece_labels_df.to_pickle('../data/wordpiece_labels_df')

In [ ]:
sentences = pd.read_pickle('../data/wordpiece_sentences_df')
labels = pd.read_pickle('../data/wordpiece_labels_df')

In [ ]:
display(sentences.head())
display(labels.head())

## Replace None with \<pad\>


In [ ]:
# The tokenizer can't deal with None. We replace it with '<pad>', which the albert tokenizer encodes as padding by encoding it as 0
# This will mean that the albert model will ignore these items, which is what we want
print(tokenizer.encode('<pad>'))
sentences.fillna('<pad>', inplace=True)
labels.fillna('<pad>', inplace=True)

## Expand tokenizer vocabulary to be able to deal with our tags
The tags aren't native to swedish ner - the model won't have a representation for B-PER for example.

So we add those manually

In [6]:
# Grab all the tags we've created
label_set = set()
for column in labels.columns:
    unique_labels = labels[column].unique()
    for label in unique_labels:
        label_set.add(label)
label_list = list(label_set)

print(f"We're working with labels {label_list}")

We're working with labels ['B-LOC/ORG', 'B-EVN', 'B-LOC/PRS', 'I-LOC/ORG', 'I-TME', 'I-ORG', 'I-PRS', 'B-ORG/PRS', 'I-PRS/WRK', 'B-O', 'B-LOC', 'B-ORG', 'I-OBJ', 'B-LOC/LOC', 'B-MSR', '[SEP]', 'I-LOC', 'B-PRS/WRK', 'I-EVN', 'B-PRS', 'B-OBJ/ORG', 'I-ORG/PRS', '<pad>', 'I-O', 'I-OBJ/ORG', 'I-MSR', 'B-OBJ', 'B-WRK', '[CLS]', 'I-WRK', 'B-TME', 'I-LOC/LOC', 'I-LOC/PRS']


In [7]:
print("Testing how the tokenizer encodes our labels right now")
encoded = tokenizer.encode(label_list, add_special_tokens=False)
print(f"Encoded version of labels with current tokenizer {encoded}")
decoded = tokenizer.decode(encoded)
print(f"Decoded version of labels with current tokenizer {decoded}")
print("Yeah that's not gonna work")

Testing how the tokenizer encodes our labels right now
Encoded version of labels with current tokenizer [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]
Decoded version of labels with current tokenizer <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>[SEP]<unk><unk><unk><unk><unk><unk><pad><unk><unk><unk><unk><unk>[CLS]<unk><unk><unk><unk>
Yeah that's not gonna work


In [8]:
#Add them
tokenizer.add_tokens(label_list)
print("Testing how the tokenizer encodes our labels right now")
encoded = tokenizer.encode(label_list, add_special_tokens=False)
print(f"Encoded version of labels with current tokenizer {encoded}")
decoded = tokenizer.decode(encoded)
print(f"Decoded version of labels with current tokenizer {decoded}")
print("Yeah that's  gonna work")


Testing how the tokenizer encodes our labels right now
Encoded version of labels with current tokenizer [50000, 50001, 50002, 50003, 50004, 50005, 50006, 50007, 50008, 50009, 50010, 50011, 50012, 50013, 50014, 3, 50015, 50016, 50017, 50018, 50019, 50020, 0, 50021, 50022, 50023, 50024, 50025, 2, 50026, 50027, 50028, 50029]
Decoded version of labels with current tokenizer B-LOC/ORG B-EVN B-LOC/PRS I-LOC/ORG I-TME I-ORG I-PRS B-ORG/PRS I-PRS/WRK B-O B-LOC B-ORG I-OBJ B-LOC/LOC B-MSR [SEP] I-LOC B-PRS/WRK I-EVN B-PRS B-OBJ/ORG I-ORG/PRS <pad> I-O I-OBJ/ORG I-MSR B-OBJ B-WRK [CLS] I-WRK B-TME I-LOC/LOC I-LOC/PRS
Yeah that's  gonna work


In [28]:
import numpy as np
# Let's save the labels as well
np.save('../data/label_list', label_list)

# An important note
Adjusting the vocabulary of the tokenizer does not translate to the albert model being able to deal with that, we need to tell the model that we've adjusted the amount of embeddings with

model.resize_token_embeddings(len(tokenizer))

After updating it

## Check how the conversion to integers looks right now
Integer conversion is needed because the model won't work with charcters - it works with numbers.

So we want each token to be represented by an id

In [ ]:
def encode_without_special_symbol(token):
    """
    I'm not savy enough to know how to apply the function with only some arguments in the pandas mapping.
    I"m sure there is a way.
    But this is fast
    
    Encodes a single token without special tokens added
    """
    return tokenizer.encode(token, add_special_tokens=False)

In [ ]:
display(s.head().applymap(encode_without_special_symbol))
display(l.head().applymap(encode_without_special_symbol))

### It seems like some tokens are split into multiple tokens. Bother, let's have a look at why that is.

In [ ]:
tokenizer.encode('sakademin')

In [ ]:
tokenizer.decode([783, 85, 625])

In [ ]:
tokenizer.decode([783, 85])

In [ ]:
tokenizer.decode([625])

Okay, so apparently the encode version isn't the best one, it actually performs tokenization by itself. Since I've already split into tokens - though apparently not the same way as this function would - I have another one to use


In [ ]:
tokenizer.convert_tokens_to_ids('sakademin')

In [10]:
# Yes, this looks better.
# The conversion takes a while though
sentences = sentences.applymap(tokenizer.convert_tokens_to_ids)
labels = labels.applymap(tokenizer.convert_tokens_to_ids)


### Create attentions masks
Attention is 0 if the token is entirely unimportant. This is useful for dealing with padding, and is part of the input to the albert model

In [19]:
def attentionmapping(x):
    if x != 0:
        return 1
    else:
        return 0

In [20]:
attention = sentences.applymap(attentionmapping)

In [22]:
# Store them for later use
#sentences.to_pickle('../data/sentence_ints')
#labels.to_pickle('../data/label_ints')
#attention.to_pickle('../data/attention_ints')

In [ ]:
# Took a break, don't want to run through all the previous code
def restart():
    import pandas as pd
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("KB/albert-base-swedish-cased-alpha")
    sentences = pd.read_pickle('../data/sentence_ints')
    labels = pd.read_pickle('../data/label_ints')
    attentions = pd.read_pickle('../data/attention_ints')
    return tokenizer, sentences, labels
tokenizer, sentences, labels, attentions = restart()

# Dealing with the huge filesize

In [ ]:
import numpy as np
import pandas as pd
#Load the data
df = pd.read_pickle("../data/sentence_labels")

#Work on a subset for testing purposes
input_df = df[['Sentence']].copy()
label_df = df[['Labels']].copy()

In [ ]:
emb1 = np.load('../data/emb1.npy', mmap_mode='r')
emb2 = np.load('../data/emb2.npy', mmap_mode='r')
emb3 = np.load('../data/emb3.npy', mmap_mode='r')
emb4 = np.load('../data/emb4.npy', mmap_mode='r')
print(emb4.shape)

In [ ]:
"""
Actually the whole embedding matrix is way too big to keep in RAM. The SUC3 embedding takes 23GB, in fact
So I can only keep 40 000 wor

The size of the final embedding matrix, nevertheless, is quite big.
In fact, the size of the whole embedding matrix takes 23gb, so I broke it up into smaller pieces

emb2 = get_embeddings_with_gpu_batch(data_tensor_matrix[20000:40000], mask_tensor_matrix[20000:40000], 50)
np.save('../data/emb2', emb2)


Don't forget to remove the memory footprint emb2 =3 between each loop
x = np.load('../data/em')
"""



# Create a file with a shape capable of holding all the embedded data.
#a = np.memmap('test.mymemmap', dtype='float64', mode='w+', shape=(74245,50,768))


In [ ]:
"""
Join the separated embedding matrices into a single one

Use nmap_mode = 'r' to NOT put it into RAM
"""
emb1 = np.load('../data/emb1.npy', mmap_mode='r')
emb2 = np.load('../data/emb2.npy', mmap_mode='r')
emb3 = np.load('../data/emb3.npy', mmap_mode='r')
emb4 = np.load('../data/emb4.npy', mmap_mode='r')
#lab1 = label_matrix[:20000]

In [ ]:
"""
We've generated embeddings batch by batch since our ram can't hold it all
Now we load those saved batches from memory into a single file
"""

def load_embedded_data_into_mmap()
    a[0:20000] = emb1
    a[20000:40000] = emb2
    a[40000:60000] = emb3
    a[60000::] = emb4

In [ ]:
emb4[1] == a[60001]

In [ ]:
df = pd.read_pickle("../data/suc3_formatted")

In [ ]:
df

# Some analysis of the dataset

In [ ]:
df = pd.read_pickle("../data/sentence_labels")
df.head()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased-ner")

# use WORD tokenization
df['SENTENCE_LENGTH'] = df.apply(lambda x: len(tokenizer.encode(x.Sentence)), axis=1)

In [ ]:
df.describe()

In [ ]:
# Total number of words

word_bag = []
for row in range (df.shape[0]):
    s = df.iloc[row].Sentence
    word_bag.extend(tokenizer.encode(s))

In [ ]:
len(set(word_bag))